# Тетрадка содержит код исследования количества банкоматов, которые придется ежедневно обслуживать, если в предыдущий день обслужили все, которые переполнятся через n дней или у которых через n дней окажется больше 2 недель с момента последней инкассации

Данные для моделирования 

    - величина % в годовых, которую банк платит за неинкассированную сумму денег в терминале - 2 (т.е. сумма за день = остаток * 2/100/365);
    - стоимость обслуживания одного терминала в случае его инкассирования = 0,01% от суммы инкассации, но не менее 100 рублей (т.е. для суммы 50000 = 100, для суммы 1500000 = 150);
    - максимально допустимая сумма денег в терминале - 1000000;
    - максимально допустимое время, в течение которого терминал можно не обслуживать - 14 дней;
    - стоимость одного броневика на день - 20000;
    - начало и конец рабочего дня броневиков - c 08:00 по 20:00 мск. времени;
    - время простоя броневика на точке (время на инкассирование) - 10 минут


Возьмём терминал 406136: 
На утро 01.09 остаток = 160000.

Если его обслужить 01.09 и забрать 160000, то остаток на вечер будет = 0, платить за фондирование 01.09 не надо. Но будут издержки = 100 руб. за обслуживание.
1.1. На утро 02.09 остаток станет = 90000. Добавятся 90000 из оборотов 01.09. 

Если его не обслужить 01.09, то на вечер 01.09 остаток будет = 160000 и 01.09 у вас возникают издержки, связанные с фондированием = 7.45. Издержек за обслуживание не будет.
2.1. На утро 02.09 остаток станет = 250000 (160000 + 90000 поступлений за 01.09).
	2.1.1. Если 02.09 обслужить терминал, то остаток на вечер = 0. Фондирование = 0. Издержки за обслуживание = 100.
	2.1.2. Если 02.09 не обслуживать терминал, то остаток на вечер = 250000. Фондирование = 13.70. Издержки за обслуживание = 0.


In [ ]:
from typing import List, Tuple, Optional, Dict
from warnings import warn

import pandas as pd
import numpy as np
from scipy.stats import norm

import matplotlib.pyplot as plt

from sklearn.cluster import KMeans

from bank_schedule.ortools_tsp import solve_tsp_ortools

from bank_schedule.data import Data, distances_matrix_from_dataframe
from bank_schedule.helpers  import calc_cartesian_coords
from bank_schedule import forecast, helpers, cluster, scheduler
from bank_schedule import plot as bsplot
from bank_schedule.constants import RAW_DATA_FOLDER, INTERIM_DATA_FOLDER

In [ ]:
def get_today_from_residuals(residuals: pd.DataFrame) -> pd.Timestamp:
    """Из датафреймов с остатками в банкоматах извлекает дату,
    за вечер которой известны эти остатки

    Args:
        residuals (pd.DataFrame): _description_

    Raises:
        ValueError: _description_

    Returns:
        pd.Timestamp: _description_
    """
    today = residuals['date'].unique()
    if today.shape[0] > 1:
        raise ValueError('More than one unique date in the residuals dataframe')
    return pd.to_datetime(today[0])


def choose_mandatory_tids(cash_data: pd.DataFrame,
                          days_to_deadline_thresh=1) -> List[int]:
    """Получает список идентификаторов банкоматов, которые нужно обслужить

    Args:
        cash_data (pd.DataFrame): _description_
        days_to_deadline_thresh (int, optional): _description_. Defaults to 2.
        norm_scale (float, optional): _description_. Defaults to 2.5.
        mandatory_rate (float, optional): _description_. Defaults to 0.7.

    Returns:
        List[int]: _description_
    """

    cond = cash_data['days_to_deadline'] <= days_to_deadline_thresh

    return cash_data.loc[cond, 'TID'].unique().tolist()


def add_days_to_deadline(cash_data: pd.DataFrame) -> pd.DataFrame:
    """Добавляет колонку days_to_deadline

    Args:
        cash_data (pd.DataFrame): _description_

    Returns:
        pd.DataFrame: _description_
    """

    cash_data = cash_data.copy()
    days_to_collecting = 14 - (cash_data['date'] - cash_data['last_collection_date']).dt.days
    days_to_overflow_thresh = (cash_data['overflow_date'] - cash_data['date']).dt.days

    cash_data['days_to_deadline'] = list(map(min, days_to_collecting, days_to_overflow_thresh))
    cash_data['days_to_deadline'] = cash_data['days_to_deadline'].astype(int)

    return cash_data


def get_optimal_route(loader: Data,
                      tids_list: List[int],
                      n_iterations: int=10) -> Tuple[List[int], List[int], float]:
    """Возвращает список банкоматов, которые нужно обслужить
    в порядке оптимальног времени в пути, времена убытия из них

    Args:
        tids_list (List[int]): список банкоматов для обслуживания
        distances_df (pd.DataFrame): датафрейм расстояний
        n_iterations (int, optional): сколько попыток построения маршрута сделать. Defaults to 10.

    Returns:
        List[int]: _description_
    """
    distances_df = loader.get_distance_matrix()

    best_sum_time = float('inf')
    best_route = []
    best_route_times = []
    for _ in range(n_iterations):
        depot = np.random.choice(tids_list)
        route, route_times, sum_time = solve_tsp_ortools(tids_list,
                                               distances_df,
                                               depot)
        if len(route) != len(set(route)):
            print(route)
            print(pd.Series(data=route).value_counts())
            raise ValueError('Какие-то точки в маршруте посещаются не один раз')

        if sum_time < best_sum_time:
            best_sum_time = sum_time
            best_route = route
            best_route_times = route_times

    return best_route, best_route_times, best_sum_time


def get_neighbours(tid: int,
                   loader: Data,
                   radius: float=15,
                   ) -> List[int]:
    """Получает соседние банкоматы для tid

    Args:
        tid (int): _description_
        loader (Data): _description_
        radius (float, optional): _description_. Defaults to 15.

    Returns:
        List[int]: _description_
    """
    distances_df = loader.get_distance_matrix()

    cond1 = distances_df['Origin_tid']==tid
    cond2 = distances_df['Destination_tid']==tid

    time_cond = distances_df['Total_Time'] < radius

    set1 = set(distances_df.loc[cond1 & time_cond, 'Destination_tid'])
    set2 = set(distances_df.loc[cond2 & time_cond, 'Origin_tid'])

    return list(set1.union(set2))


def get_weights_from_residuals(residuals: pd.DataFrame,
                               std: float=7.0) -> Dict:
    """Получает веса для банкоматов по их близости к дедлайну
    (по обслуживанию и переполнению)

    Args:
        residuals (pd.DataFrame): _description_

    Returns:
        Dict: _description_
    """
    weights_base = residuals[['TID', 'days_to_deadline']].drop_duplicates()
    mynnorm = norm(0, std)
    weights_base['weight'] = weights_base['days_to_deadline'].apply(mynnorm.pdf)
    return weights_base.set_index('TID')['weight'].to_dict()


def add_random_neighbours(tids_list: List[int],
                         loader: Data,
                         radius: int,
                         n_neighbours: int,
                         tids_weights: Optional[Dict[int, float]]=None,
                         ) -> List[int]:
    """Докидываем соседей к точкам обязательной инкассации

    Args:
        route (List[int]): _description_
        loader (Data): _description_
        radius (int): _description_
        n_neighbours (int): _description_

    Raises:
        ValueError: _description_

    Returns:
        List[int]: _description_
    """
    if n_neighbours <= 0:
        warn('Соседей не понадобилось')
        return tids_list

    all_neighbours = []

    for tid in tids_list:
        all_neighbours += get_neighbours(tid, loader, radius)

    all_neighbours = list(
        set(all_neighbours).difference(set(tids_list))
        )

    if not all_neighbours:
        warn('Соседей не нашлось')
        return tids_list

    n_nb = len(all_neighbours)

    if n_nb < n_neighbours:
        warn(f'Найдено только {n_nb} соседей, это меньше {n_neighbours}. Все они будут добавлены в маршрут.')
        n_neighbours = n_nb

    probas = None

    if tids_weights is not None:
        probas = [tids_weights[tid] for tid in all_neighbours]
        probas_sum = sum(probas)
        probas = [prb / probas_sum for prb in probas]

    selected = list(np.random.choice(all_neighbours,
                                     n_neighbours,
                                     replace=False,
                                     p=probas))

    if len(selected) != len(set(selected)):
        raise ValueError('Соседние точки задублились')

    return tids_list + selected


def split_oneroute(route: List[int],
                   route_time: List[float],
                   max_route_time: float):
    """_summary_

    Args:
        route (List[int]): _description_
        route_time (List[float]): _description_
        max_route_time (float): _description_
    """
    pass


def find_n_cars(loader: Data,
                tids_list: List[int],
                atms_per_day: int=150,
                n_iterations: int=1,
                radius: float=15,
                max_route_time: float=720.0,
                tids_weights: Optional[Dict[int, float]]=None):
    """Находит количество машин, необходимое для обслуживания банкоматов
    в день

    Args:
        loader (Data): _description_
        tids_list (List[int]): _description_
        max_tids (int, optional): _description_. Defaults to 70.
        atms_per_day (int, optional): _description_. Defaults to 150.
        n_iterations (int, optional): _description_. Defaults to 1.
        radius (float, optional): _description_. Defaults to 15.
        max_route_time (float, optional): _description_. Defaults to 720.0.
        tids_weights (Optional[Dict[int, float]], optional): _description_. Defaults to None.
        allowed_percent (float, optional): _description_. Defaults to 0.05.

    Returns:
        _type_: _description_
    """
    n_neighbours = atms_per_day - len(tids_list)
    extended_tids = add_random_neighbours(tids_list,
                                          loader,
                                          radius,
                                          n_neighbours,
                                          tids_weights=tids_weights)

    route, route_time, route_time_sum = get_optimal_route(loader,
                                                          extended_tids,
                                                          n_iterations=n_iterations)

    return route, route_time, route_time_sum / max_route_time


def optimize_routes(loader: Data,
                    tids_list: List[int],
                    atms_per_day: int=200,
                    n_iterations: int=1,
                    radius: float=15,
                    max_route_time: float=720.0,
                    tids_weights: Optional[Dict[int, float]]=None) -> Tuple[List[int], List[List[int]], List[List[int]]]:
    """Оптимизирует маршруты инкассации

    Args:
        loader (Data): _description_
        tids_list (List[int]): _description_
        max_tids (int, optional): _description_. Defaults to 70.
        atms_per_day (int, optional): _description_. Defaults to 150.
        n_iterations (int, optional): _description_. Defaults to 1.
        radius (float, optional): _description_. Defaults to 15.
        max_route_time (float, optional): _description_. Defaults to 720.0.
        tids_weights (Optional[Dict[int, float]], optional): _description_. Defaults to None.
    """
    n_neighbours = atms_per_day - len(tids_list)
    extended_tids = add_random_neighbours(tids_list,
                                          loader,
                                          radius,
                                          n_neighbours,
                                          tids_weights=tids_weights)

    # вычисляем оптимальный маршрут
    route, route_time, route_time_sum = get_optimal_route(loader,
                                                          extended_tids,
                                                          n_iterations=n_iterations)

    if route_time_sum / max_route_time > max_route_time:
        warn(f'Превышено допустимое время: {route_time} > {max_route_time}')

    return route, route_time, route_time_sum / max_route_time


def line_route_to_points_pairs(route: List[int]) -> List[Tuple[int, int]]:
    """Преобразует маршрут в массивов точеепар из двоездов

    Args:
        route (List[int]): _description_

    Returns:
        List[Tuple[int, int]]: _description_
    """
    pairs = []
    for i in range(len(route) - 1):
        pairs.append((route[i], route[i + 1]))
    return pairs


In [ ]:
loader = Data(RAW_DATA_FOLDER)
dists = loader.get_distance_matrix()
geo = loader.get_geo_TIDS()

income = loader.get_money_in()
residuals = loader.get_money_start()

residuals['date'] = pd.to_datetime('2022-08-31')
residuals = add_last_cash_collection_date(residuals, income, fake=True)

coords_df = helpers.calc_cartesian_coords(lat_series=geo['latitude'],
                                          lon_series=geo['longitude'])
coords_df = geo.join(coords_df)

In [ ]:
route = [683103, 634763, 636538]
nearest = add_random_neighbours(route, loader, radius=10, n_neighbours=50)

to_plot = coords_df.loc[coords_df['TID'].isin(nearest + route), :].copy()
to_plot['main_pair'] = to_plot['TID'].isin(route)

bsplot.geoplot_clusters(to_plot, 'main_pair', html_folder=INTERIM_DATA_FOLDER)

In [ ]:
hist_mdl = forecast.ForecastHistorical()
lgbm_mdl = forecast.IncomeForecastLGBM()

In [ ]:
atms_for_today_collection = scheduler.get_atms_for_today_collection(new_residuals,
                                                                    mandatory_selection_threshold=1,
                                                                    mandatory_selection_col='days_to_deadline',
                                                                    tids_col='TID')

In [ ]:
# Оценим теоретически достижимое максимальное число точек,
# которое может обслужить одна машина за день
sort_dist = dists.sort_values(by='Total_Time')

# прибавляем ко времени пути время на инкассацию
sort_dist['Total_Time'] += 10

# считаем куммулятивную сумму так, как будто все близкие точки попали в один кластер
# и мы их последовательно инкассируем
sort_dist['cumsum_time'] = sort_dist['Total_Time'].cumsum()

# смотрим, сколько точек мы успели объехать
max_tids = sort_dist[sort_dist['cumsum_time'] < 12 * 60].shape[0]
print('Максимальное теоретически достижимое число точек на 1 машину:', max_tids)

new_residuals = residuals.copy()
end_date = pd.to_datetime('2022-09-14')
today = get_today_from_residuals(new_residuals)

dates, routes, routes_times, n_cars, n_atms = [], [], [], [], []

horizon = 14
deadline_thresh = 1
n_iterations = 1
max_route_time = 720
atms_per_day = 220
radius = 15

while today < end_date:
    dates.append(today)
    tomorrow = today + pd.Timedelta(days=1)

    new_residuals = add_overflow_date(new_residuals,
                                      forecast_model=lgbm_mdl,
                                      horizon=horizon)

    new_residuals = add_days_to_deadline(new_residuals)

    tids_to_collect = choose_mandatory_tids(new_residuals,
                                            days_to_deadline_thresh=deadline_thresh)

    tids_weights = get_weights_from_residuals(new_residuals)

    day_routes, day_routes_times, day_n_cars = find_n_cars(loader,
                                                            tids_to_collect,
                                                            atms_per_day=atms_per_day,
                                                            n_iterations=n_iterations,
                                                            radius=radius,
                                                            max_route_time=max_route_time,
                                                            tids_weights=tids_weights)

    n_cars.append(day_n_cars)
    routes.append(day_routes)
    routes_times.append(day_routes_times)

    print(today, len(day_routes), day_n_cars)

    n_atms.append( len(tids_to_collect) )

    # обнуляем остатки и дату в инкассированных банкоматах
    collected_cond = new_residuals['TID'].isin(day_routes)
    new_residuals['date'] = tomorrow
    new_residuals.loc[collected_cond, 'money'] = 0
    new_residuals.loc[collected_cond, 'last_collection_date'] = tomorrow
    new_residuals.loc[collected_cond, 'overflow_date'] = pd.NaT

    # считаем остаток на вечер дня инкассации, зная income за этот день
    resid_money = new_residuals.set_index('TID')['money']
    income_money = income.set_index('TID')
    income_money = income_money.loc[income_money['date']==tomorrow, 'money_in']
    income_money = income_money[resid_money.index]

    new_residuals['money'] = (resid_money + income_money).values
    today = tomorrow

In [ ]:
routes_series = pd.Series(index=dates, data=n_atms)

fig, ax = plt.subplots(figsize=(12, 6))
plt.plot(pd.Series(data=routes_series, index=dates))
plt.title('Банкоматы с deadline_soon при инкассации в предыдущий день\n'
          f'всех банкоматов с deadline_soon (маркер - меньше {deadline_thresh} дней до дедлайна)\n'
          f'дедлайн - меньше {deadline_thresh} дней до переполнения или больше {13-deadline_thresh} с прошлой инкассации')
fig.autofmt_xdate()
plt.tight_layout()
plt.ylim(0,300)
plt.grid()
plt.show()

cars_series = pd.Series(index=dates, data=n_cars)

fig, ax = plt.subplots(figsize=(12, 6))
plt.plot(pd.Series(data=cars_series, index=dates))
plt.title('Количество машин')
fig.autofmt_xdate()
plt.tight_layout()
plt.ylim(0,10)
plt.grid()
plt.show()

In [ ]:
# Оценим теоретически достижимое максимальное число точек,
# которое может обслужить одна машина за день
sort_dist = dists.sort_values(by='Total_Time')

# прибавляем ко времени пути время на инкассацию
sort_dist['Total_Time'] += 10

# считаем куммулятивную сумму так, как будто все близкие точки попали в один кластер
# и мы их последовательно инкассируем
sort_dist['cumsum_time'] = sort_dist['Total_Time'].cumsum()

# смотрим, сколько точек мы успели объехать
max_tids = sort_dist[sort_dist['cumsum_time'] < 12 * 60].shape[0]
print('Максимальное теоретически достижимое число точек на 1 машину:', max_tids)

new_residuals = residuals.copy()
end_date = pd.to_datetime('2022-11-30')
today = get_today_from_residuals(new_residuals)

dates, routes, routes_times, n_cars, n_atms = [], [], [], [], []

horizon = 14
deadline_thresh = 1
n_iterations = 1
max_route_time = 720
atms_per_day = 200
radius = 15

while today < end_date:
    dates.append(today)
    tomorrow = today + pd.Timedelta(days=1)

    new_residuals = add_overflow_date(new_residuals,
                                      forecast_model=lgbm_mdl,
                                      horizon=horizon)

    new_residuals = add_days_to_deadline(new_residuals)

    tids_to_collect = choose_mandatory_tids(new_residuals,
                                            days_to_deadline_thresh=deadline_thresh)
    day_n_atms = len(tids_to_collect)
    day_routes = [tids_to_collect]

    tids_weights = get_weights_from_residuals(new_residuals)

    day_routes, day_routes_times, day_n_cars = optimize_routes(loader,
                                                               tids_to_collect,
                                                               atms_per_day=atms_per_day,
                                                               n_iterations=n_iterations,
                                                               radius=radius,
                                                               max_route_time=max_route_time,
                                                               tids_weights=tids_weights)

    n_cars.append(day_n_cars)
    routes.append(day_routes)
    routes_times.append(day_routes_times)

    print(today, len([item for sublist in day_routes for item in sublist]), day_n_cars)

    n_atms.append(day_n_atms)

    # обнуляем остатки и дату в инкассированных банкоматах
    collected_cond = new_residuals['TID'].isin([item for sublist in day_routes for item in sublist])
    new_residuals['date'] = tomorrow
    new_residuals.loc[collected_cond, 'money'] = 0
    new_residuals.loc[collected_cond, 'last_collection_date'] = tomorrow
    new_residuals.loc[collected_cond, 'overflow_date'] = pd.NaT

    # считаем остаток на вечер дня инкассации, зная income за этот день
    resid_money = new_residuals.set_index('TID')['money']
    income_money = income.set_index('TID')
    income_money = income_money.loc[income_money['date']==tomorrow, 'money_in']
    income_money = income_money[resid_money.index]

    new_residuals['money'] = (resid_money + income_money).values
    today = tomorrow

In [ ]:
day = 1
geoplot_df = geo.copy()
geoplot_df['was_collected'] = geoplot_df.index.isin(day_routes_times[day])
bsplot.geoplot_clusters(geoplot_df,
                        'was_collected')

In [ ]:
import plotly.graph_objects as go

fig = go.Figure()

fig.add_trace(go.Scattergeo(
    locationmode = 'USA-states',
    lon = df_airports['long'],
    lat = df_airports['lat'],
    hoverinfo = 'text',
    text = df_airports['airport'],
    mode = 'markers',
    marker = dict(
        size = 2,
        color = 'rgb(255, 0, 0)',
        line = dict(
            width = 3,
            color = 'rgba(68, 68, 68, 0)'
        )
    )))

lons = []
lats = []
import numpy as np
lons = np.empty(3 * len(df_flight_paths))
lons[::3] = df_flight_paths['start_lon']
lons[1::3] = df_flight_paths['end_lon']
lons[2::3] = None
lats = np.empty(3 * len(df_flight_paths))
lats[::3] = df_flight_paths['start_lat']
lats[1::3] = df_flight_paths['end_lat']
lats[2::3] = None

fig.add_trace(
    go.Scattergeo(
        locationmode = 'USA-states',
        lon = lons,
        lat = lats,
        mode = 'lines',
        line = dict(width = 1,color = 'red'),
        opacity = 0.5
    )
)

fig.update_layout(
    title_text = 'Feb. 2011 American Airline flight paths<br>(Hover for airport names)',
    showlegend = False,
    geo = go.layout.Geo(
        scope = 'north america',
        projection_type = 'azimuthal equal area',
        showland = True,
        landcolor = 'rgb(243, 243, 243)',
        countrycolor = 'rgb(204, 204, 204)',
    ),
    height=700,
)

fig.show()